In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import cv2
import glob 
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout


In [11]:
data_entry = pd.read_csv('Data_Entry_2017.csv')
all_imgs = glob.glob("dataset/*.png")
images = []

mem_scale =  20
selection_range = int(len(all_imgs)/mem_scale)

for x in range(selection_range):
    images.append(cv2.imread(all_imgs[x]))
    image_dim = (512,512)
    images[x] = cv2.resize(np.array(images[x]), image_dim)

In [13]:
y = data_entry.iloc[:selection_range, 1]
X = images

In [14]:
for i in range(len(y)):
    if y[i] == 'No Finding':
        y[i] = 0
    else:
        y[i] = 1
        
y = pd.to_numeric(y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 1)

In [16]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,  zoom_range = 0.2, horizontal_flip = True)
training_set = train_datagen.flow(x = X_train, y = y_train, batch_size = 32)

In [18]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow(x = X_test, y = y_test, batch_size = 32)

In [19]:
nets = 1
model = [0] * nets

for j in range(nets):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=[512, 512, 3]))
    model[j].add(MaxPool2D(padding = 'valid', strides = 2))
    model[j].add(Dropout(j*0.1))
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D(padding = 'valid', strides = 2))
    model[j].add(Dropout(j*0.1))
    model[j].add(Flatten())
    model[j].add(Dense(128, activation='relu'))
    model[j].add(Dropout(j*0.1))
    model[j].add(tf.keras.layers.Dense(units=218, activation='relu'))
    model[j].add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    model[j].compile(optimizer="adam", loss = 'binary_crossentropy', metrics=["accuracy"])
    model[j].fit(x = training_set, validation_data = test_set, epochs = 1)

141/141 [==============================] - 154s 1s/step - loss: 0.9000 - accuracy: 0.5491 - val_loss: 0.6895 - val_accuracy: 0.5428


In [ ]:
# Part 2 - Building the CNN
'''
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[1024, 1024, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=4, strides=2))

# Adding second and third convolutional layers
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 5)'''

In [42]:
# Part 4 - Making a single prediction

test_image = cv2.imread(all_imgs[selection_range])
image_dim = (512,512)
test_image = cv2.resize(test_image, image_dim)
test_image = np.array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model[0].predict(test_image)


if result[0][0] == 1:
    prediction = 'pathology'
else:
    prediction = 'no pathology'


print('Real:' + data_entry.iloc[selection_range + 1, 1])
print('Print:' + prediction)

cv2.imshow('Test Image', cv2.imread(all_imgs[selection_range + 1]))
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image


Real:Nodule
Print:no pathology
